# Join French Crop Usage (FCU) with TAXREF-LD via GEVES scientific names

Join FCU with TAXREF-LD using GEVES as an intermediate because GEVES has most FCU varieties and provides scientific names.
Process:
- join FCU with GEVES: where FCU variety name = GEVES species name
- join only the matched GEVES species names with the scientific names in TAXREF-LD

### Initializations

In [35]:
import json
import os
from string import Template
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON, POST
from time import sleep
from math import isnan, nan

In [36]:
# Initializations
prefixes = '''
prefix api:            <http://ns.inria.fr/sparql-micro-service/api#>
prefix fcu:            <http://ontology.inrae.fr/frenchcropusage/>
prefix owl:            <http://www.w3.org/2002/07/owl#>
prefix rdf:            <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs:           <http://www.w3.org/2000/01/rdf-schema#>
prefix skos:           <http://www.w3.org/2004/02/skos/core#>
prefix skosxl:         <http://www.w3.org/2008/05/skos-xl#>
prefix taxrefp:        <http://taxref.mnhn.fr/lod/property/>
prefix taxrefrk:       <http://taxref.mnhn.fr/lod/taxrank/>
prefix xsd:            <http://www.w3.org/2001/XMLSchema#>
'''

In [37]:
local_endpoint = 'http://localhost:8080/sparql'
fcu_endpoint = "http://ontology.inrae.fr/frenchcropusage/sparql"
taxref_endpoint = "https://taxref.mnhn.fr/sparql"
geves_endpoint = "http://graph.i3s.unice.fr/repositories/geves"

In [38]:
def exec_sparql(endpoint, query):
    """
    Execute a SPARQL query and return results as a Pandas DataFrame.
    Invocation uses the HTTP POST method. Requested result format is JSON.

    In case of failure, the function retries up to MAX_ATTEMPTS attempts while waiting increasing time between each attempt.
    An empty DataFrame is returned in case no attempt is successful.
    """
    # Max number of time a SPARQL query can fail before giving up
    MAX_ATTEMPTS = 3

    sparql = SPARQLWrapper(endpoint)
    sparql.setMethod(POST)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    
    attempt = 1;
    success = False
    while not success and attempt <= MAX_ATTEMPTS:
        try:
            raw_results = sparql.query()
            results = raw_results.convert()
        except:
            print(f'Error while executing SPARQL query (attempt {attempt}/{MAX_ATTEMPTS}).', end=' ')
            if attempt < MAX_ATTEMPTS:
                # Wait a few seconds before next attempt (5, 10, 15, ...)
                print(f'Will retry in {attempt * 5}s.')
                sleep(attempt * 5)
        else:
            success = True
        finally:
            attempt = attempt + 1

    if success:
        if 'head' in results and 'results' in results:
            cols = results['head']['vars']
            out = []
            for row in results['results']['bindings']:
                item = []
                for c in cols:
                    item.append(row.get(c, {}).get('value'))
                out.append(item)
            return pd.DataFrame(out, columns=cols)
        else:
            print('Invalid SPARQL result. Will return empty DataFrame.\n' + str(results))
    else:
        print(f'Unable to execute SPARQL query after {MAX_ATTEMPTS} attempts. Will return empty DataFarme.')
                  
    return pd.DataFrame()            


def dataframe_preview(df, start=0, end=10):
    print("== Number of lines: " + str(df.shape[0]))
    print("== Number of unique values:")
    print(df.nunique())
    display(df[start:end])

---
## Join FCU with GEVES: FCU varieties names = GEVES species

In [41]:
query =  prefixes + '''
select distinct ?fcu_concept ?fcu_name ?fcu_name_type ?geves_lab_spe_dus ?geves_scientific_name where {

    # Query FCU crops
    service <http://ontology.inrae.fr/frenchcropusage/sparql> {
        select distinct ?fcu_concept ?fcu_name ?fcu_name_type where {

            # Select only cultivated crops
            <http://ontology.inrae.fr/frenchcropusage/Usages_plantes_cultivees> skos:narrower+ ?fcu_concept.

            # Keep only leaves or their direct parent but not above
            FILTER (NOT EXISTS { ?fcu_concept skos:narrower/skos:narrower ?child. })

            # Get prefered and alternate labels and keep track of the type of label
            ?fcu_concept a skos:Concept.
            { ?fcu_concept skos:prefLabel ?lb. bind("pref" as ?fcu_name_type) }
            UNION
            { ?fcu_concept skos:altLabel  ?lb. bind("alt" as ?fcu_name_type) }

            bind(str(lcase(?lb)) as ?fcu_name)
        } order by ?fcu_concept
    }

    # Query GEVES
    optional {
        service <http://graph.i3s.unice.fr/repositories/geves> {
            [] 
                api:lab_spe_bota ?lab_spe_bota;
                api:lab_spe_dus ?geves_lab_spe_dus.
            bind(str(lcase(?lab_spe_bota))      as ?geves_scientific_name)
            bind(str(lcase(?geves_lab_spe_dus)) as ?geves_species)
        }

        # Match the GEVES species names and the FCU preferred/alternate labels:
        filter (?fcu_name = ?geves_species) 
    }
}'''

In [42]:
%time df_fcu_geves = exec_sparql(local_endpoint, query)

Wall time: 13.4 s


In [43]:
dataframe_preview(df_fcu_geves, end=10)

== Number of lines: 1521
== Number of unique values:
fcu_concept               446
fcu_name                 1481
fcu_name_type               2
geves_lab_spe_dus          89
geves_scientific_name      85
dtype: int64


,fcu_concept,fcu_name,fcu_name_type,geves_lab_spe_dus,geves_scientific_name
0,http://ontology.inrae.fr/frenchcropusage/Arbre...,arbre fruitier à noyau,pref,None,None
1,http://ontology.inrae.fr/frenchcropusage/Arbre...,arbre à noyau,alt,None,None
2,http://ontology.inrae.fr/frenchcropusage/Arbre...,fruit à noyau,alt,None,None
3,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier,pref,Abricotier,prunus armeniaca l.
4,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot,alt,None,None
5,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier pays,pref,None,None
6,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot pays,alt,None,None
7,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier des antilles,alt,None,None
8,http://ontology.inrae.fr/frenchcropusage/Abric...,mamey,alt,None,None
9,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier-pays,alt,None,None


In [44]:
#df_fcu_geves.to_csv("result1_fcu_geves.csv", index=False)
df_fcu_geves.to_excel("result1_fcu_geves.xlsx")
#df_fcu_geves = pd.read_csv("result1_fcu_geves.csv")

#### Count only matches

In [45]:
dataframe_preview(df_fcu_geves.dropna())

== Number of lines: 98
== Number of unique values:
fcu_concept              91
fcu_name                 89
fcu_name_type             2
geves_lab_spe_dus        89
geves_scientific_name    85
dtype: int64


,fcu_concept,fcu_name,fcu_name_type,geves_lab_spe_dus,geves_scientific_name
3,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier,pref,Abricotier,prunus armeniaca l.
37,http://ontology.inrae.fr/frenchcropusage/Ceris...,cerisier acide,alt,Cerisier acide,prunus cerasus l.
38,http://ontology.inrae.fr/frenchcropusage/Ceris...,cerisier doux,alt,Cerisier doux,prunus avium l.
39,http://ontology.inrae.fr/frenchcropusage/Ceris...,cerisier doux,alt,Cerisier doux,prunus cerasus l.
54,http://ontology.inrae.fr/frenchcropusage/Pechers,pêcher,pref,Pêcher,prunus persica (l.) batsch
62,http://ontology.inrae.fr/frenchcropusage/Pruniers,prunier,pref,Prunier,prunus domestica l.
153,http://ontology.inrae.fr/frenchcropusage/Cassi...,cassis,alt,Cassis,ribes nigrum l.
154,http://ontology.inrae.fr/frenchcropusage/Framb...,framboisier,pref,Framboisier,rubus idaeus l.
156,http://ontology.inrae.fr/frenchcropusage/Grose...,groseillier,pref,Groseillier,ribes rubrum l.
181,http://ontology.inrae.fr/frenchcropusage/Citro...,citronnier,pref,Citronnier,citrus limon (l.) burm.


### Matches:
- FCU: 91 unique concepts, 89 unique labels
- GEVES: 89 unique species, 85 unique scientific names

---
## Join GEVES with TAXREF-LD on scientific name, only on the GEVES species matched with FCU

In [46]:
# SPARQL query to TAXREF-LD to match a single GEVES scientific name (placeholder $geves_scientific_name)
queryTpl = Template(prefixes + '''
    select distinct ("$geves_scientific_name" as ?geves_scientific_name) ?taxref_full_name ?taxref_name_type ?taxon ?rank 
    from <http://taxref.mnhn.fr/lod/graph/classes/13.0>
    from <http://taxref.mnhn.fr/lod/graph/concepts/13.0>
    where {
        ?name
           a                      skos:Concept, <http://rs.tdwg.org/ontology/voc/TaxonName#TaxonName>;
           rdfs:label             ?taxref_full_name.
           
        { ?name taxrefp:isReferenceNameOf ?taxon. bind("pref" as ?taxref_name_type) }
        union
        { ?name taxrefp:isSynonymOf       ?taxon. bind("alt" as ?taxref_name_type) }

        ?taxon
           taxrefp:hasRank        ?rank.

        # All ranks up to spcecies but not above
        filter (?rank in (
            taxrefrk:Species,  taxrefrk:SemiSpecies, taxrefrk:MicroSpecies, taxrefrk:SubSpecies, taxrefrk:Natio, 
            taxrefrk:Varietas, taxrefrk:SubVarietas, taxrefrk:Forma,        taxrefrk:SubForma,   taxrefrk:FormaSpecies,
            taxrefrk:Linea,    taxrefrk:Clone,       taxrefrk:Race,         taxrefrk:Cultivar,   taxrefrk:Morpha,
            taxrefrk:Abberatio ))
        
        # TAXREF names have the authority and date, whereas GEVES names do not have the date.
        # => match the GEVES name as a subpart of the TAXREF name
        bind(str(lcase(?taxref_full_name)) as ?taxref_scn_sl)
        filter(strstarts(?taxref_scn_sl, "$geves_scientific_name"))
    }
''')

In [47]:
# Set a max number of queries to submit. 0 = unlimited.
MAX_QUERIES = 0

# Result DataFrame
df_geves_taxref = pd.DataFrame()

idx = 1
df_fcu_geves_matched = df_fcu_geves.dropna()
unique_names = df_fcu_geves_matched.geves_scientific_name.unique()
for geves_scientific_name in unique_names:
    query = queryTpl.substitute(geves_scientific_name = geves_scientific_name.strip().lower())
    #print(query)
    
    print(f"---- Running query {idx}/{len(unique_names)} - geves_scientific_name = {geves_scientific_name}")
    %time _df = exec_sparql(taxref_endpoint, query)
    print(f'Number of results: {_df.shape[0]}')
    df_geves_taxref = df_geves_taxref.append(_df)
    
    # Keep track of GEVES names not matched with TAXREF
    if _df.shape[0] == 0:
        nomatch_row = {'taxref_full_name': None, 'taxref_name_type': None, 'taxon': None, 'geves_scientific_name': geves_scientific_name.strip().lower()}
        df_geves_taxref = df_geves_taxref.append(nomatch_row, ignore_index=True)
    
    idx = idx + 1
    if MAX_QUERIES > 0 and idx > MAX_QUERIES:
        break

---- Running query 1/85 - geves_scientific_name = prunus armeniaca l.
Wall time: 14.4 s
Number of results: 1
---- Running query 2/85 - geves_scientific_name = prunus cerasus l.
Wall time: 14.3 s
Number of results: 1
---- Running query 3/85 - geves_scientific_name = prunus avium l.
Wall time: 13.7 s
Number of results: 0
---- Running query 4/85 - geves_scientific_name = prunus persica (l.) batsch
Wall time: 12.8 s
Number of results: 1
---- Running query 5/85 - geves_scientific_name = prunus domestica l.
Wall time: 11.7 s
Number of results: 1
---- Running query 6/85 - geves_scientific_name = ribes nigrum l.
Wall time: 12.3 s
Number of results: 1
---- Running query 7/85 - geves_scientific_name = rubus idaeus l.
Wall time: 12.3 s
Number of results: 1
---- Running query 8/85 - geves_scientific_name = ribes rubrum l.
Wall time: 14.3 s
Number of results: 1
---- Running query 9/85 - geves_scientific_name = citrus limon (l.) burm.
Wall time: 13.1 s
Number of results: 1
---- Running query 10/85 -

In [48]:
#df_geves_taxref.to_csv("result2_geves_taxref.csv", index=False)
df_geves_taxref.to_excel("result2_geves_taxref.xlsx")
#df_geves_taxref = pd.read_csv("result2_geves_taxref.csv")

In [49]:
dataframe_preview(df_geves_taxref)

== Number of lines: 87
== Number of unique values:
geves_scientific_name    85
taxref_full_name         59
taxref_name_type          2
taxon                    57
rank                      2
dtype: int64


,geves_scientific_name,taxref_full_name,taxref_name_type,taxon,rank
0,prunus armeniaca l.,"Prunus armeniaca L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116041/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
1,prunus cerasus l.,"Prunus cerasus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116054/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
2,prunus avium l.,None,None,None,NaN
3,prunus persica (l.) batsch,"Prunus persica (L.) Batsch, 1801",pref,http://taxref.mnhn.fr/lod/taxon/116112/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
4,prunus domestica l.,"Prunus domestica L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116067/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
5,ribes nigrum l.,"Ribes nigrum L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/117766/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
6,rubus idaeus l.,"Rubus idaeus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/119149/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
7,ribes rubrum l.,"Ribes rubrum L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/117774/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
8,citrus limon (l.) burm.,"Citrus limon (L.) Burm.f., 1768",pref,http://taxref.mnhn.fr/lod/taxon/91809/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
9,citrus clementina hort. ex. tan.,None,None,None,NaN


#### Count only matches

In [50]:
dataframe_preview(df_geves_taxref.dropna())

== Number of lines: 59
== Number of unique values:
geves_scientific_name    57
taxref_full_name         59
taxref_name_type          2
taxon                    57
rank                      2
dtype: int64


,geves_scientific_name,taxref_full_name,taxref_name_type,taxon,rank
0,prunus armeniaca l.,"Prunus armeniaca L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116041/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
1,prunus cerasus l.,"Prunus cerasus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116054/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
3,prunus persica (l.) batsch,"Prunus persica (L.) Batsch, 1801",pref,http://taxref.mnhn.fr/lod/taxon/116112/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
4,prunus domestica l.,"Prunus domestica L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116067/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
5,ribes nigrum l.,"Ribes nigrum L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/117766/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
6,rubus idaeus l.,"Rubus idaeus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/119149/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
7,ribes rubrum l.,"Ribes rubrum L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/117774/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
8,citrus limon (l.) burm.,"Citrus limon (L.) Burm.f., 1768",pref,http://taxref.mnhn.fr/lod/taxon/91809/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
10,citrus deliciosa ten.,"Citrus deliciosa Ten., 1840",alt,http://taxref.mnhn.fr/lod/taxon/91812/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
14,citrus medica l.,"Citrus medica L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/91811/13.0,http://taxref.mnhn.fr/lod/taxrank/Species


### Matches:
- GEVES: 57 unique scientific names
- TAXREF-LD: 57 unique taxa

=> 32 GEVES names not matched with TAXREF-LD

---
## Join intermediate results: FCU-GEVES and GEVES-TAXREFLD

In [51]:
df_merge = pd.merge(df_fcu_geves, df_geves_taxref, on="geves_scientific_name", how='left')
df_merge.drop_duplicates(inplace=True)
dataframe_preview(df_merge)

== Number of lines: 1523
== Number of unique values:
fcu_concept               446
fcu_name                 1481
fcu_name_type               2
geves_lab_spe_dus          89
geves_scientific_name      85
taxref_full_name           59
taxref_name_type            2
taxon                      57
rank                        2
dtype: int64


,fcu_concept,fcu_name,fcu_name_type,geves_lab_spe_dus,geves_scientific_name,taxref_full_name,taxref_name_type,taxon,rank
0,http://ontology.inrae.fr/frenchcropusage/Arbre...,arbre fruitier à noyau,pref,None,None,NaN,NaN,NaN,NaN
1,http://ontology.inrae.fr/frenchcropusage/Arbre...,arbre à noyau,alt,None,None,NaN,NaN,NaN,NaN
2,http://ontology.inrae.fr/frenchcropusage/Arbre...,fruit à noyau,alt,None,None,NaN,NaN,NaN,NaN
3,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier,pref,Abricotier,prunus armeniaca l.,"Prunus armeniaca L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116041/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
4,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot,alt,None,None,NaN,NaN,NaN,NaN
5,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier pays,pref,None,None,NaN,NaN,NaN,NaN
6,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot pays,alt,None,None,NaN,NaN,NaN,NaN
7,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier des antilles,alt,None,None,NaN,NaN,NaN,NaN
8,http://ontology.inrae.fr/frenchcropusage/Abric...,mamey,alt,None,None,NaN,NaN,NaN,NaN
9,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier-pays,alt,None,None,NaN,NaN,NaN,NaN


In [52]:
#df_merge.to_csv("result3_fcu_geves_taxref.csv", index=False)
df_merge.to_excel("result3_fcu_geves_taxref.xlsx")
#df_merge = pd.read_csv("result3_fcu_geves_taxref.csv")

#### Count only matches

In [53]:
dataframe_preview(df_merge.dropna())

== Number of lines: 70
== Number of unique values:
fcu_concept              64
fcu_name                 63
fcu_name_type             2
geves_lab_spe_dus        63
geves_scientific_name    57
taxref_full_name         59
taxref_name_type          2
taxon                    57
rank                      2
dtype: int64


,fcu_concept,fcu_name,fcu_name_type,geves_lab_spe_dus,geves_scientific_name,taxref_full_name,taxref_name_type,taxon,rank
3,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier,pref,Abricotier,prunus armeniaca l.,"Prunus armeniaca L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116041/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
37,http://ontology.inrae.fr/frenchcropusage/Ceris...,cerisier acide,alt,Cerisier acide,prunus cerasus l.,"Prunus cerasus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116054/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
39,http://ontology.inrae.fr/frenchcropusage/Ceris...,cerisier doux,alt,Cerisier doux,prunus cerasus l.,"Prunus cerasus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116054/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
54,http://ontology.inrae.fr/frenchcropusage/Pechers,pêcher,pref,Pêcher,prunus persica (l.) batsch,"Prunus persica (L.) Batsch, 1801",pref,http://taxref.mnhn.fr/lod/taxon/116112/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
62,http://ontology.inrae.fr/frenchcropusage/Pruniers,prunier,pref,Prunier,prunus domestica l.,"Prunus domestica L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116067/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
153,http://ontology.inrae.fr/frenchcropusage/Cassi...,cassis,alt,Cassis,ribes nigrum l.,"Ribes nigrum L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/117766/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
154,http://ontology.inrae.fr/frenchcropusage/Framb...,framboisier,pref,Framboisier,rubus idaeus l.,"Rubus idaeus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/119149/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
156,http://ontology.inrae.fr/frenchcropusage/Grose...,groseillier,pref,Groseillier,ribes rubrum l.,"Ribes rubrum L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/117774/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
181,http://ontology.inrae.fr/frenchcropusage/Citro...,citronnier,pref,Citronnier,citrus limon (l.) burm.,"Citrus limon (L.) Burm.f., 1768",pref,http://taxref.mnhn.fr/lod/taxon/91809/13.0,http://taxref.mnhn.fr/lod/taxrank/Species
192,http://ontology.inrae.fr/frenchcropusage/Manda...,mandarinier,pref,Mandarinier,citrus deliciosa ten.,"Citrus deliciosa Ten., 1840",alt,http://taxref.mnhn.fr/lod/taxon/91812/13.0,http://taxref.mnhn.fr/lod/taxrank/Species


### Matches
- FCU: 64 unique concepts, 63 unique labels
- TAXREF-LD: 57 unique taxa from 2 ranks